In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [2]:
from govrfp.rfpinsights._chromadb import return_collection, add_entries_to_collection
path = "tests/mar-19/data/"
critical_collection = return_collection(path=path, collection_name="critical")
informative_collection = return_collection(path=path, collection_name="informative")

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
# read lines from "tests/mar-19/01-compliances.txt"
with open("tests/mar-19/01-compliances.txt", "r") as file:
    compliances = file.readlines()
    compliances = [compliance.strip() for compliance in compliances]
compliances[:5]

['Email DE-FOA-0003120@netl.doe.gov for questions about the FOA',
 'Email EERE-eXCHANGE Support@hq.doe.gov for issues with EERE eXCHANGE, include FOA name and number in the subject line',
 'Submit a Concept Paper by 5:00 PM ET on the due date to be eligible for a Full Application',
 'Register and submit application materials through EERE eXCHANGE at https://eere-eXCHANGE.energy.gov/',
 'Designate primary and backup points-of-contact in EERE eXCHANGE for communication during award negotiations']

In [4]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [5]:
# return top n matching compliances
def get_matching_compliances(critical_collection, query, n=5):
    res = critical_collection.query(query_texts=[query], n_results=n)['documents'][0]
    if len(res) == 0:
        return "Empty"
    return "\n".join([f'- {t}' for t in res])

In [6]:
prompt = """As an AI language model, your task is to accurately classify compliance items from a Request for Proposal (RFP) application. Assign a label of 0 to a compliance item if it is critical, mandatory, or directly impacts the application's functionality. This could be a hard requirement, soft requirement, or an explicitly defined requirement. Label it as 1 if it is optional, meaning it is not necessary but could enhance the application or is applicable in certain cases. Phrases like "if applicable" or "upon request" often indicate optional items. Assign -1 if the item is not a compliance requirement or if it is a duplicate of an item already listed. Be mindful of the context, implications, and nuances of the compliance items to accurately classify them. Pay attention to edge cases, such as duplicate items or items that may seem critical but are actually optional based on their implications.

Attached list of compliance items:
{compliance_items}

Compliance Item:
{compliance_item}

Return 0, 1, or -1:"""
print(prompt.format(compliance_items=get_matching_compliances(critical_collection, compliances[0], n=10), compliance_item=compliances[0]))

As an AI language model, your task is to accurately classify compliance items from a Request for Proposal (RFP) application. Assign a label of 0 to a compliance item if it is critical, mandatory, or directly impacts the application's functionality. This could be a hard requirement, soft requirement, or an explicitly defined requirement. Label it as 1 if it is optional, meaning it is not necessary but could enhance the application or is applicable in certain cases. Phrases like "if applicable" or "upon request" often indicate optional items. Assign -1 if the item is not a compliance requirement or if it is a duplicate of an item already listed. Be mindful of the context, implications, and nuances of the compliance items to accurately classify them. Pay attention to edge cases, such as duplicate items or items that may seem critical but are actually optional based on their implications.

Attached list of compliance items:
Empty

Compliance Item:
Email DE-FOA-0003120@netl.doe.gov for ques

In [7]:
llm.complete(prompt.format(compliance_items=get_matching_compliances(critical_collection, compliances[0], 10), compliance_item=compliances[0])).text

'-1'

In [8]:
import tqdm
import time
critical_compliance = []
informative_compliance = []
other_compliance = []
rating = []

In [9]:
#  import something to cler the output
from IPython.display import clear_output

In [10]:
for index, compliance in tqdm.tqdm(enumerate(compliances)):
    # clear the output
    clear_output(wait=True)
    print(len(critical_compliance), len(informative_compliance), len(other_compliance))
    try:
        response = llm.complete(prompt.format(compliance_items=get_matching_compliances(critical_collection, compliance, n=15), compliance_item=compliance)).text
    except:
        continue
    try:
        _rating = int(response)
        rating.append(_rating)
    except:
        if '-1' in response:
            _rating = -1
            rating.append(_rating)
        elif '0' in response:
            _rating = 0
            rating.append(_rating)
        elif '1' in response:
            _rating = 1
            rating.append(_rating)
        else:
            print(response)
            _rating = int(input("Enter the correct rating: "))
            rating.append(_rating)
    if _rating == 0:
        add_entries_to_collection(
            docs=[compliance],
            metadata=[{'id': f"{index}"}],
            ids=[f"{index}"],
            collection=critical_collection
        )
        critical_compliance.append(compliance)
    elif _rating == 1:
        add_entries_to_collection(
            docs=[compliance],
            metadata=[{'id': f"{index}"}],
            ids=[f"{index}"],
            collection=informative_collection
        )
        informative_compliance.append(compliance)
    else:
        other_compliance.append(compliance)


937 362 150


1459it [18:01,  1.35it/s]


In [11]:
# export critical compliaces
with open("tests/mar-19/new_critical_compliances.txt", "w") as file:
    file.write("\n".join(critical_compliance))

# export informative compliaces
with open("tests/mar-19/new_informative_compliances.txt", "w") as file:
    file.write("\n".join(informative_compliance))

# export other compliaces
with open("tests/mar-19/new_other_compliances.txt", "w") as file:
    file.write("\n".join(other_compliance))